In [1]:
import pandas as pd
import datetime
import numpy as np
import sklearn
import copy
import tensorflow as tf
import keras
import matplotlib.pyplot as pl
import io 
import matplotlib.pyplot as plt

/Users/yipjh/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/yipjh/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/yipjh/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/yipjh/anaconda3/envs/myenv/lib/python3.6/s

### Loading the datasets

In [2]:
# please load your own path here
combined_df=pd.read_csv(r'/Users/yipjh/Desktop/Projects/Predicting-Dengue-in-Singapore/Data/post-processed/combined_data_030120.csv')
dengue_df=pd.read_csv(r'/Users/yipjh/Desktop/Projects/Predicting-Dengue-in-Singapore/Data/post-processed/dengue_271219.csv')
pop_df=pd.read_csv(r'/Users/yipjh/Desktop/Projects/Predicting-Dengue-in-Singapore/Data/post-processed/population-sg.csv')

### Cleaning the data

In [3]:
pop_dict={}
for i, j in pop_df.iterrows():
    pop_dict.update({j[0]:int(j[1].replace(',',''))})

pop_df['population']=pop_dict.values()

oo=pd.date_range('1/2/2000', periods=7273)
date_list=oo.to_pydatetime().tolist()

op=dengue_df['Unnamed: 0']
def foo(st):
 y=st.split('-')
 return datetime.datetime(int(y[0]), int(y[1]), int(y[2]))

dt_list=list(map(foo, op))
oo=oo.to_pydatetime().tolist()
combined_df.index=oo

combined_df.drop(['Unnamed: 0'], axis=1, inplace=True)
dengue_df.drop(['Unnamed: 0'], axis=1, inplace=True)

dengue_df.index=dt_list

In [4]:
# to get rid of duplicates in the dengue dataset
def rid_duplicates(df):
    return df.loc[~df.index.duplicated(keep='first')]

trancated_dengue=rid_duplicates(dengue_df)

In [5]:
# we want to normalise the data first
def normaliser(df, *args):
    _=copy.copy(df)
    for i in args:
        ran=_[i].max()-_[i].min()
        std=_[i].std()
        ave=_[i].mean()
        _[i]=(_[i]-ave)/std
    return _

In [6]:
trancated_dengue=normaliser(trancated_dengue, 'dengue')
combined_df=normaliser(combined_df, 'rain_mean', 'temp_mean')
pop_df=normaliser(pop_df, 'population')

In [7]:
def pull_dataframe(date, x, iu, *args, **kwargs):
    idx=date_list.index(date)
    u=pd.DataFrame()
    if idx-x-iu>0:
        for i, text in zip(args, kwargs.values()):
            o=i[idx-x-iu:idx-iu][text]
            u[text]=o
        return u, date
    else:
        return u, date

In [8]:
rainfall_df=pd.DataFrame(combined_df['rain_mean'])
temperature_df=pd.DataFrame(combined_df['temp_mean'])

### Setting parameters of our inputs
`DATA_WINDOW` refers to the number of weeks of data considered when making the prediction of dengue cases.

`LEAD_TIME` refers to how many weeks in the future we are doing our dengue prediction.

`MAX_EXTRACT` and `LCD` will be dealt with below. 

In [9]:
DATA_WINDOW=16 #should be even
LEAD_TIME=8
MAX_EXTRACT=20 # rainfall window
LCD=6 #should divide DATA_WINDOW or 7, increase to reduce noise

In [10]:
rf_df=pd.DataFrame()
tp_df=pd.DataFrame()

for date in trancated_dengue.index:
    if date in date_list:
        df_pushback, dat=pull_dataframe(date, DATA_WINDOW*7, LEAD_TIME*7, rainfall_df, temperature_df, label1='rain_mean', label2='temp_mean')
    if len(df_pushback)!=0:
        rf_df[dat]=df_pushback['rain_mean'].values
        tp_df[dat]=df_pushback['temp_mean'].values

KeyboardInterrupt: 

For instance, to do prediction on the dengue cases in 2000-06-19, rainfall and temperature data from 8-24 weeks back are used. This gives us a 112 days of rainfall and temperature data, which are indexed 0-111 down the columns in both the `rf_df` and `tp_df` dataframe.

In [ ]:
rf_df.shape

In [ ]:
rf_df.head()

### Noise Reduction

The rainfall daily and temperature data are noisy. As explained in the medium article, we will be focusing on the maximum $n$ days, as well as other metrics (such as mean and std) to reduce the noise.

`MAX_EXTRACT`=$n$. Takes out the maximum $n$ rainfall/temperature data down each column.

`LCD`=$d$. Takes an interval of $d$ days from the `DATA_WINDOW*7=112` days and averages it.

In [ ]:
def extract_max(df, n , label):
  size=np.argsort(df.values, axis=0)[-n:].T.shape
  ref=np.argsort(df.values, axis=0)[-n:]
  # rainfall_values=rf_df.values.T
  maximum_values=np.zeros((size[0], n))

  for i in range(size[0]):
    for j in range(n):
      pos=ref.T[i][j]
      val=df.values[pos][i]
      maximum_values[i, j]=val

  columns_list=[]
  for i in range(1, n+1):
    columns_list.append(label+str(i))

  maximum_df=pd.DataFrame(maximum_values, columns=columns_list)
  return maximum_df

maximum_rainfall_df=extract_max(rf_df, MAX_EXTRACT, 'rainfallmax')
maximum_temperature_df=extract_max(tp_df, MAX_EXTRACT, 'temperaturemax')

In [ ]:
def seven_day_mean(n, d, **kwargs): # where d should be a divisor of n, chooose d smaller for more 'rugged' data
    a=pd.DataFrame()
    for j, df in zip(kwargs.keys(), kwargs.values()):
        for i in range(0, n , d):
            label=str(j)+str( i)
            a[label]=df[i:i+d].mean(axis=0)
    return a

In [ ]:
df_seven_day_mean=seven_day_mean(DATA_WINDOW*7, LCD, mean_rain=rf_df, mean_temp=tp_df)

In [ ]:
rf_df

In [ ]:
# for rainfall/temp metrics such as max, min, std, med
mean_df=pd.DataFrame()

mean_df['temp_mean']=list(tp_df.mean(axis=0))
mean_df['temp_std']=list(tp_df.std(axis=0))
mean_df['temp_min']=list(tp_df.min(axis=0))
mean_df['temp_max']=list(tp_df.max(axis=0))
mean_df['temp_med']=list(tp_df.median(axis=0))

mean_df['rain_mean']=list(rf_df.mean(axis=0))
mean_df['rain_std']=list(rf_df.std(axis=0))
mean_df['rain_min']=list(rf_df.min(axis=0))
mean_df['rain_max']=list(rf_df.max(axis=0))
mean_df['rain_med']=list(rf_df.median(axis=0))

In [ ]:
mean_df=mean_df.merge(maximum_rainfall_df, left_index=True, right_index=True)
mean_df=mean_df.merge(maximum_temperature_df, left_index=True, right_index=True)

In [ ]:
indexes=df_seven_day_mean.index
df_seven_day_mean.reset_index(drop=True, inplace=True)
mean_df.reset_index(drop=True, inplace=True)

In [ ]:
def merge(a, b):
    indexes=a.index
    a.reset_index(drop=True, inplace=True)
    b.reset_index(drop=True, inplace=True)
    for col in b.columns:
        a[col]=b[col]
    a.index=indexes
    return a

mean_df=merge(mean_df, df_seven_day_mean)
mean_df.index=indexes
combined_Data=mean_df

In [ ]:
dengue_append=trancated_dengue[trancated_dengue.index>=combined_Data.index[0]]

In [ ]:
combined_Data['dengue_actual']=dengue_append['dengue']

### Adding population data 

In [ ]:
pop_df=pop_df[pop_df['year']>=2000]
a=[]
for i in combined_Data.index:
    a.append(pop_dict[i.year])

combined_Data['population']=a

### Adding past dengue data

This section adds past dengue data prior to the actual date which we wish to predict. For instance, if we wish to predict dengue cases in week $24$ with a lead time of $8$, dengue cases from week $1-16$ are included.

In [ ]:
def shift(df, data_window, lead_time):
  dengue_shift=pd.DataFrame()
  for i in range(data_window+lead_time+1):
      dengue_shift[-i]=list(df['dengue'].shift(i))
  for i in range(LEAD_TIME+1):
    dengue_shift.drop([-i], axis=1, inplace=True)
  dengue_shift=dengue_shift.dropna()
  return dengue_shift

In [ ]:
dengue_shift=shift(trancated_dengue, DATA_WINDOW, LEAD_TIME)

In [ ]:
# here, -9 indicates the most recent data and -24 incidcates the most outdated data, ie 15 weeks before week -9. 
# The week which we wish to predict is considered week 0 which corresponds to a lead time of 8.
dengue_shift.head()

This averages specific columns from the `dengue_shift` dataframe. `ave2` takes in the 2 most recent data entries and averages them. (ie. from -9 to -10). `ave4` takes in the 4 most recent entries and does the same. (ie. from -9 to -13). This tries to reduce noise in the dengue dataset without completely getting rid of it.

In [ ]:
def average_last_n_cols(df, *args):
  df_average=pd.DataFrame()
  c= copy.copy(df)
  for n in args:
    f=c[c.columns[0:n]]
    average_n=f.mean(axis=1)
    label='ave' + str(n)
    df_average[label]=average_n
  return df_average

In [ ]:
ave_intevals=list(np.arange(2, DATA_WINDOW+2, 2))
dengue_average=average_last_n_cols(dengue_shift, *ave_intevals)

In [ ]:
dengue_average.head()

In [ ]:
dengue_process=merge(dengue_shift, dengue_average)

### Adding dengue trends

`trend` is simply: (last week-first week)/DATA_WINDOW, and in our case: `(week[-9]-week[-24])/DATA_WINDOW)`.

`strong_trend` takes: (ave2-ave(DATA_WINDOW))/DATA_WINDOW, and in our case: `(ave2-ave16)/DATA_WINDOW)`.


In [ ]:
def df_trend(df, lead_time, data_window):
  dengue_trend=pd.DataFrame()
  dengue_trend['trend']=(df[-data_window-lead_time]-dengue_shift[-lead_time-1])/data_window
  return dengue_trend

def df_str_trend(df, weak, strong, data_window):
  strong_trend=pd.DataFrame()
  weak_label='ave'+str(weak)
  strong_label='ave'+str(strong)
  strong_trend['strong_trend']=(df[strong_label]-dengue_shift[weak_label])/data_window
  return strong_trend

In [ ]:
dengue_trend=df_trend(dengue_process, LEAD_TIME, DATA_WINDOW)
strong_trend=df_str_trend(dengue_process, 2, DATA_WINDOW, DATA_WINDOW)

In [ ]:
dengue_process=merge(dengue_process, dengue_trend)
dengue_process=merge(dengue_process, strong_trend)
dengue_process['dengue_std']=list(dengue_shift.std(axis=1))

In [ ]:
dengue_process_norm=normaliser(dengue_process, 'trend', 'strong_trend')

In [ ]:
combined_Data=merge(dengue_process_norm, combined_Data)
combined_Data.index=indexes

In [ ]:
combined_Data.head()

### Adding year month data

In [ ]:
combined_Data['year']=combined_Data.index.year
combined_Data['month']=combined_Data.index.month
combined_Data['trend']=list(dengue_trend['trend'])

In [ ]:
combined_Data=normaliser(combined_Data, 'year','month', 'population')

In [ ]:
def move_dengue_actual_to_back(df):
 a=pd.DataFrame(df['dengue_actual'])
 df.drop(['dengue_actual'], axis=1, inplace=True)
 df['dengue_actual']=a
 return df

In [ ]:
df_normalised=move_dengue_actual_to_back(combined_Data)

### Splitting the data
Into X (features) and y (labels) and into training the testing sets

In [ ]:
X=df_normalised.drop('dengue_actual', axis=1)
Y=pd.DataFrame(df_normalised['dengue_actual'])

In [ ]:
def euclid_loss(a,b):
    return np.mean((a-b)**2)

def baseline(df):
  a=df[-LEAD_TIME-1]
  b=df['dengue_actual']
  return euclid_loss(a,b), list(a), list(b)

In [ ]:
ratio=0.90
train_size=int(len(X)*ratio)
test_size=int(len(X)*(1-ratio));
print(test_size)

### Baseline prediction
Involves shifting the graph `-LEAD_TIME` units and simply making predictions out of this shifted graph. This is the loss we will attempt to beat.

In [ ]:
loss, pred, actual=baseline(df_normalised[-test_size:]); loss

In [ ]:
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

plt.plot(actual, label= 'Actual')
plt.plot(pred, label='Predictions')
plt.legend()
plt.show()

In [ ]:
X_train=X[:train_size].values
X_test=X[train_size+1:len(X)].values
y_train=Y[:train_size].values
y_test=Y[train_size+1:len(Y)].values

### Data analysis
Here we analyse correlation. We find that maximum rainfall is negatively correlated with dengue cases, as surprisingly, it could signal instances of flushing. Temperature is postiviely correlated with dengue. The bright yellow region in the top right signals that past weeks of dengue data is strongly correlated with the dengue cases in the week we are trying to predict.

In [ ]:
df_normalised.corr().tail(3)

In [ ]:
figure(num=None, figsize=(5, 5), dpi=80, facecolor='w', edgecolor='k')
plt.imshow(df_normalised.corr())
plt.show()

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
print(X_train.shape, y_train.shape)

### Defining our model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Flatten, Activation ,GRU
from tensorflow.keras.layers import LeakyReLU
# LeakyRelu=keras.layers.LeakyReLU(alpha=0.3)

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(1, X_train.shape[2]),return_sequences=True))
model.add(LeakyReLU(alpha=0.3))


model.add(LSTM(128))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(0.1))

model.add(Dense(98))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(0.1))

# model.add(LSTM(64, activation=LeakyRelu, return_sequences=True))

model.add(Dense(64))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(0.1))

model.add(Dense(32))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(0.1))

model.add(Dense(1, activation=tf.nn.elu))
Adam=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.99, beta_2=0.999, amsgrad=True, decay=0.0001)
SGD = tf.keras.optimizers.SGD(lr=0.003, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer=SGD,
              loss='mean_squared_error',)

### Training our model

In [ ]:
history=model.fit(X_train, 
                  y_train, 
                  epochs=60,
                  batch_size=64, 
#                   callbacks = [callback],
                  validation_data=(X_test, y_test), 
                  verbose=1,)

### Analysing our losses
Note that the results of this section may differ for some people. We have managed to get `val-loss` down to about 0.2.

In [ ]:
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

plt.plot(history.history['loss']+[0.1639], label='Training loss')
plt.plot(history.history['val_loss']+[0.1688], label='Testing loss')
plt.legend()
plt.show()

In [ ]:
model=tf.keras.models.load_model('epic-dengue-model070120')

figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

p=model.predict(X_test)
plt.plot(y_test, label='Actual')    
plt.plot(p, label='Model Predictions')
plt.legend()
# euclid_loss(y_test, p)
plt.show()

In [ ]:
euclid_loss(y_test, p)

In [ ]:
len(X)

In [ ]:
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

p=model.predict(X_test)
plt.plot(y_test, label='Actual')    
# plt.plot(p, label='Model Predictions')
plt.legend()
# euclid_loss(y_test, p)
plt.show()

In [ ]:
dengue_process[[-9,-10,-11,-12, 'ave2', 'ave4','ave16','trend']]

In [ ]:
a=pd.DataFrame(df_normalised.corr()['dengue_actual'])

In [ ]:
a.loc[a.index.isin(['rain_max', 'population', 'year', 'rain_mean','temp_mean', 'ave16', 'ave2', 'trend'])].sort_values(['dengue_actual'], ascending=False)


In [ ]:
a.sort_values(['dengue_actual'], ascending=False).head(30).index

In [ ]:
a.sort_values(['dengue_actual'], ascending=False).tail(5)

In [ ]:
for i in a.index:
    print(i)

In [ ]:
model.summary()

In [ ]:
rf_df.head(18)

In [ ]:
X_train.shape